In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm


from codes.appendix import SectCode
from codes.appendix import FatrCode

In [41]:
from IPython.core.display import HTML

# 글씨 크기를 10px로 조절하는 CSS
css = """
<style>
table.dataframe td, table.dataframe th {
    font-size: 10px;
}
</style>
"""
HTML(css)

In [42]:
from src.api import SmartFarmAPI

api = SmartFarmAPI(config="config.yaml")

In [132]:
class TomatoDataProcessor:
    def __init__(self, serl_nos):
        self.serl_nos = serl_nos
        self.all_page_data = []


    def get_data_for_page(self, serl_no,page):
        return pd.DataFrame(api.crop_season.getCroppingSeasonEnvDataList(serl_no, page))
        
        
    def get_data_all_pages(self, serl_no):
        first_page_data=self.get_data_for_page(serl_no,1)
        total_page = int(float(first_page_data.at[0, 'totalPage']))

        all_data=[]
        for page in range(1, total_page+2):
            page_data = self.get_data_for_page(serl_no, page)
            page_data['SerlNo'] = serl_no
            all_data.append(page_data)
        return pd.concat(all_data)

    def transform_data(self, final_df):
        df_selected = final_df[["measDate", "fldCode", "sectCode", "fatrCode", "senVal","SerlNo"]]
        df_selected["senVal"] =  pd.to_numeric(df_selected["senVal"], errors='coerce')
        df_selected["measDate"] = pd.to_datetime(df_selected["measDate"])
        df_pivoted = df_selected.pivot_table(index=['measDate', 'SerlNo'], columns='fatrCode', values='senVal', aggfunc='mean')
        df_pivoted.rename(columns=lambda x: FatrCode.get_description(x), inplace=True)
        return df_pivoted

    def run(self):
        for no in self.serl_nos:
            all_page_data = self.get_data_all_pages(no)
            self.all_page_data.append(all_page_data)

        final_df = pd.concat(self.all_page_data, ignore_index=True)
        transformed_df = self.transform_data(final_df)
        return transformed_df

if __name__ == "__main__":
    df = pd.DataFrame(api.crop_season.getCroppingSeasonDataList("2021"))
    df_tomato = df[(df["itemName"] == "토마토") & (df["acqAutoYn"] == "Y")]
    df_tomato['croppingSerlNo'].unique()
    serl = np.array([4419,4420,4430])
    processor = TomatoDataProcessor(serl)
    df_pivoted = processor.run()
    print(df_pivoted)
 



        상세 기능 번호  : 1
        상세 기능 설명  : 스마트팜 보급 농가 사용자의 작기 정보 조회 
        
        * 요청 메세지 명세 *    
            year: 작기 시작 년도 String (형식 : yyyy)
            
        * 응답 메세지 명세 *
            facilityId: 시설 ID String 
            addressName: 지역명 String (법정동명)
            croppingSerlNo: 작기 일련번호 Integer 
            itemCode       : 품목코드 String 
            itemName: 품목명 String
            croppingDate: 작기 시작 일자 String (형식 : yyyy-mm-dd)
            croppingEndDate: 작기 종료 일자 String (형식 : yyyy-mm-dd)
            acqAutoYn: 환경정보 등록 유무 String (Y : 등록, N : 미등록)
            acqManlYn: 제어정보 등록 유무 String (Y : 등록, N : 미등록)
            acqCultiYn: 생육정보 등록 유무 String (Y : 등록, N : 미등록)
            acqMgmtYn : 경영정보 등록 유무 String (Y : 등록, N : 미등록)
        

        * Response *
            - statusCode    : 00
            - statusMessage : NORMAL_CODE
            - statusDesc    : 정상
        

        상세 기능 번호  : 2
        상세 기능 설명  : 스마트팜 보급 농가 사용자의 작기별 환경 정보 조회
        
        * 요청 메세지 명세 *   

C:\Users\mirun\AppData\Local\Temp\ipykernel_30476\4030536902.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["senVal"] =  pd.to_numeric(df_selected["senVal"], errors='coerce')
C:\Users\mirun\AppData\Local\Temp\ipykernel_30476\4030536902.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["measDate"] = pd.to_datetime(df_selected["measDate"])


In [133]:
df_pivoted

,measDate,fldCode,sectCode,fatrCode,senVal,SerlNo
0,2021-01-12 00:00:00,FG,EI,CI,606.17,4419
1,2021-01-12 00:00:00,FG,EI,HI,92.77,4419
2,2021-01-12 00:00:00,FG,EI,TI,15.33,4419
3,2021-01-12 00:00:00,FG,EO,TE,-3.91,4419
4,2021-01-12 01:00:00,FG,EI,HI,95.14,4419
...,...,...,...,...,...,...
99847,2021-07-04 23:00:00,FG,EI,TI,15.48,4430
99848,2021-07-04 23:00:00,FG,EL,TL,24.18,4430
99849,2021-07-04 23:00:00,FG,EO,RP,0.25,4430
99850,2021-07-04 23:00:00,FG,EO,TE,24.00,4430
